Downloads description of articles
Cargo un archivo en forma de dataframe, exitraigo un sub-dataframe, scrapeo el comentario de wallapop y lo retorno en un df que es id - descripcion

In [3]:
import os
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime

In [4]:
elemento_a_buscar = "iphone 15" # para guardar el archivo
iphone = 15
# 'new', 'as_good_as_new', 'good'
#Solo uno cada vez!
estado = 'new'
carpeta = "3_feature_engineering"
fecha = datetime.now().strftime("%Y%m%d")
#fecha = "20241114"

In [5]:
# Cargo archivo

df_original = pd.read_excel(carpeta + '/' + fecha + '.xlsx')
df_original.head()


,id,fecha,gen,mod,memoria,bateria,precio,estado,nombre,reservado,url,tiene_color,tiene_emojis,tiene_revisado
0,1073642029,20241127,15,basic,NaN,NaN,"590,00",as_good_as_new,iphone 15,False,https://es.wallapop.com/item/iphone-15-1073642029,False,False,False
1,1073757376,20241127,15,basic,128,NaN,"599,00",good,iphone 15 128gb negro,False,https://es.wallapop.com/item/iphone-15-128gb-n...,True,False,False
2,1073789336,20241127,15,pro,NaN,NaN,"899,00",new,iphone 15 pro,False,https://es.wallapop.com/item/iphone-15-pro-107...,False,False,False
3,1073756454,20241127,15,plus,NaN,NaN,"800,00",new,iphone 15 plus a estrenar,False,https://es.wallapop.com/item/iphone-15-plus-a-...,False,False,False
4,1073765761,20241127,15,pro,128,NaN,"799,00",good,iphone 15 pro 128gb natural,False,https://es.wallapop.com/item/iphone-15-pro-128...,False,False,False


In [6]:
# Selecciono el sub-dataframe.
# Selecciono:       as_good_as_new 
#                   iphone 15. 
#                   bateria = NaN
# Son 263 apariciones
df_to_scrap = df_original[(df_original['estado'] == estado) & 
                          (df_original['gen'] == iphone) & 
                          (pd.isna(df_original['bateria']))]
df_to_scrap.reset_index(drop=True, inplace = True)
df_to_scrap.head(5)

,id,fecha,gen,mod,memoria,bateria,precio,estado,nombre,reservado,url,tiene_color,tiene_emojis,tiene_revisado
0,1073789336,20241127,15,pro,NaN,NaN,"899,00",new,iphone 15 pro,False,https://es.wallapop.com/item/iphone-15-pro-107...,False,False,False
1,1073756454,20241127,15,plus,NaN,NaN,"800,00",new,iphone 15 plus a estrenar,False,https://es.wallapop.com/item/iphone-15-plus-a-...,False,False,False
2,1073716209,20241127,15,pro max,256,NaN,"999,00",new,iphone 15 pro max 256 gb nuevo,False,https://es.wallapop.com/item/iphone-15-pro-max...,False,False,False
3,1073000950,20241127,15,pro max,256,NaN,"995,00",new,iphone 15 pro max 256gb reacondicionado,False,https://es.wallapop.com/item/iphone-15-pro-max...,False,False,False
4,1064233981,20241127,15,basic,128,NaN,"635,00",new,iphone 15 rosa 128gb nuevo,False,https://es.wallapop.com/item/iphone-15-rosa-12...,True,False,False


In [7]:
# Función de scrapeo comentarios
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# no existe https://es.wallapop.com/item/iphone-15-128gb-como-nuevo-1069439182
# si existe https://es.wallapop.com/item/iphone-15-pro-max-esim-256gb-1069441441

In [8]:
import time
# 263 anuncios - 25 min. con sleep de 1
# Escrapeo objtos de:
# Vendedor, cuadro datos, cuadro envio, cuadro comentario, cuadro 'editado views y likes'
# por ahora solo comentario
def scrap_comments(df):
    df_len = len(df)
    df['comentario'] = None
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.get(df.loc[0]['url']) 
    WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "onetrust-reject-all-handler"))).click() # quita cookies y carga primera web
    for index in range(len(df)):
        driver.get(df.loc[index]['url']) 
        print(index, '/', df_len)
        try:                                                                                    # si existe la web
            elemento_comentario = driver.find_element(By.CSS_SELECTOR, "#__next > main > div > div.item-detail_ItemDetail__container__8p25r.pb-5 > section.justify-content-around.d-flex.py-4 > div.item-detail_ItemDetail__card__jnUEv.pb-5 > div.px-4 > div.item-detail_ItemDetail__separator__SCH3p.py-2 > section").text
            df.loc[index, 'comentario'] = elemento_comentario
            print(df.loc[index]['url'], ': \n', df.loc[index]['comentario'])                                                                                     
        except:                                                                              # Si no existe la web: 
            element = driver.find_element(By.CLASS_NAME, "not-found-page_Error__title__Dky_8")
            elemento_comentario = "not found"
            df.loc[index, 'comentario'] = elemento_comentario
            print(df.loc[index]['url'], df.loc[index, 'comentario'])
        #time.sleep(1)
    driver.quit()
    return (df)

df_with_comments = scrap_comments(df_to_scrap)

C:\Users\ACER\AppData\Local\Temp\ipykernel_9224\4260173602.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comentario'] = None


0 / 139
https://es.wallapop.com/item/iphone-15-pro-1073789336 : 
 Vendo iphone 15 pro, color titanio natural.128gb. Reemplazado por apple hace 5 dias. Totalmente nuevo. Con su caja, garantia 2ños. Applecare+ con robo y perdida. 
Solo trato en persona.
1 / 139
https://es.wallapop.com/item/iphone-15-plus-a-estrenar-1073756454 : 
 Vendo iphone 15 plus 256gb , color negro , precintado (tu lo estrenas) . No entrego factura , puedo enseñarla para la verificación del producto sin problema . La garantía se activa encuanto se active por primera vez el telefono . 
NO CAMBIO , ABTENERSE OFERTAS ABSURDAS , NO VAS A ENCONTRAR OTRO A ESTE PRECIO Y SIN ABRIR . Un saludo y buen dia
2 / 139
https://es.wallapop.com/item/iphone-15-pro-max-256-gb-nuevo-1073716209 : 
 Phone 15 Pro Max de 256 gb
Con apenas uso , sin el mas minimo rasguño , con 1 años de garantia y factura 
Se puede probar en persona

( Se aceptan dispositivos como forma de pago )

ℹ️Información dispositivo ℹ️
- ﻿﻿Memoria: 256 Gb
- ﻿﻿Estado 

In [9]:
df_with_comments.head(4)

,id,fecha,gen,mod,memoria,bateria,precio,estado,nombre,reservado,url,tiene_color,tiene_emojis,tiene_revisado,comentario
0,1073789336,20241127,15,pro,NaN,NaN,"899,00",new,iphone 15 pro,False,https://es.wallapop.com/item/iphone-15-pro-107...,False,False,False,"Vendo iphone 15 pro, color titanio natural.128..."
1,1073756454,20241127,15,plus,NaN,NaN,"800,00",new,iphone 15 plus a estrenar,False,https://es.wallapop.com/item/iphone-15-plus-a-...,False,False,False,"Vendo iphone 15 plus 256gb , color negro , pre..."
2,1073716209,20241127,15,pro max,256,NaN,"999,00",new,iphone 15 pro max 256 gb nuevo,False,https://es.wallapop.com/item/iphone-15-pro-max...,False,False,False,"Phone 15 Pro Max de 256 gb\nCon apenas uso , s..."
3,1073000950,20241127,15,pro max,256,NaN,"995,00",new,iphone 15 pro max 256gb reacondicionado,False,https://es.wallapop.com/item/iphone-15-pro-max...,False,False,False,Vendo Iphone 15 Pro Max 256GB reacondicionado ...


In [11]:
subdf_comentarios = df_with_comments[['id', 'comentario']]
subdf_comentarios

,id,comentario
0,1073789336,"Vendo iphone 15 pro, color titanio natural.128..."
1,1073756454,"Vendo iphone 15 plus 256gb , color negro , pre..."
2,1073716209,"Phone 15 Pro Max de 256 gb\nCon apenas uso , s..."
3,1073000950,Vendo Iphone 15 Pro Max 256GB reacondicionado ...
4,1064233981,Iphone 15 rosa nuevo.\n128GB \n93% salud de ba...
...,...,...
134,1010823700,Somos una tienda online de electrónica y hogar...
135,1010823027,Somos una tienda online de electrónica y hogar...
136,981673881,apple iphone 15 Plus 256Gb CF001_E19149_0 es u...
137,1015943533,i phone15 de 256gb nuevo con 2 meses de uso de...


In [12]:
# Guardo comentarios finales en el df original
df_original = pd.merge(df_original, subdf_comentarios, on='id', how='left')


In [13]:
df_original.head(5)

,id,fecha,gen,mod,memoria,bateria,precio,estado,nombre,reservado,url,tiene_color,tiene_emojis,tiene_revisado,comentario
0,1073642029,20241127,15,basic,NaN,NaN,"590,00",as_good_as_new,iphone 15,False,https://es.wallapop.com/item/iphone-15-1073642029,False,False,False,NaN
1,1073757376,20241127,15,basic,128,NaN,"599,00",good,iphone 15 128gb negro,False,https://es.wallapop.com/item/iphone-15-128gb-n...,True,False,False,NaN
2,1073789336,20241127,15,pro,NaN,NaN,"899,00",new,iphone 15 pro,False,https://es.wallapop.com/item/iphone-15-pro-107...,False,False,False,"Vendo iphone 15 pro, color titanio natural.128..."
3,1073756454,20241127,15,plus,NaN,NaN,"800,00",new,iphone 15 plus a estrenar,False,https://es.wallapop.com/item/iphone-15-plus-a-...,False,False,False,"Vendo iphone 15 plus 256gb , color negro , pre..."
4,1073765761,20241127,15,pro,128,NaN,"799,00",good,iphone 15 pro 128gb natural,False,https://es.wallapop.com/item/iphone-15-pro-128...,False,False,False,NaN


In [14]:
nombre_archivo_csv = df_original
carpeta = "4_download_description"
nombre_archivo_pkl = fecha + '_' + elemento_a_buscar + '_' + estado
df_original.to_csv(carpeta + '/' + nombre_archivo_pkl + '.csv')